In [8]:
import os
import pandas as pd
import datetime
import urllib.request
import json
import requests, json

In [9]:
df_station = pd.read_csv('All_station.csv')
df_station

,id,location,name
0,02t,แขวงหิรัญรูจี เขตธนบุรี กทม.,มหาวิทยาลัยราชภัฏบ้านสมเด็จเจ้าพระยา
1,05t,แขวงบางนา เขตบางนา กทม.,กรมอุตุนิยมวิทยา
2,10t,แขวงคลองจั่น เขตบางกะปิ กทม.,เคหะชุมชนคลองจั่น
3,11t,แขวงดินแดง เขตดินแดง กทม.,สนามกีฬาเคหะชุมชนห้วยขวาง
4,12t,แขวงช่องนนทรี เขตยานนาวา กทม.,โรงเรียนนนทรีวิทยา
...,...,...,...
82,63t,ต.สะเตง อ.เมือง จ.ยะลา,สวนสาธารณะสนามโรงพิธีช้างเผือก ยะลา
83,78t,ต.เบตง อ.เบตง จ.ยะลา,ศูนย์พัฒนาเด็กเล็กเทศบาลเมืองเบตง
84,80t,ต.พิมาน อ.เมือง. จ.สตูล,ศาลากลางจังหวัดสตูล
85,89t,ต.คลัง อ.เมือง จ.นครศรีธรรมราช,ศูนย์ฟื้นฟูสุขภาพผู้สูงอายุ


In [10]:
station_id = df_station.id.tolist()
type(station_id)

list

In [11]:
stations = station_id
measurements=['PM10', 'O3', 'CO', 'NO2', 'SO2', 'WS', 'WD', 'TEMP', 'RH', 'BP', 'RAIN']
sdate='2023-01-01'
edate='2024-01-02'

In [12]:
URL = 'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}' +  \
     '&param={measurements}&type=hr&sdate={sdate}&edate={edate}'+ \
     '&stime=00&etime=24'
URL

'http://air4thai.pcd.go.th/webV2/history/api/data.php?stationID={station_ids}&param={measurements}&type=hr&sdate={sdate}&edate={edate}&stime=00&etime=24'

In [13]:
station_groups = [stations[i:i+5] for i in range(0, len(stations), 5)]
station_groups

[['02t', '05t', '10t', '11t', '12t'],
 ['59t', '61t', '03t', '50t', '52t'],
 ['53t', '54t', '08t', '16t', '17t'],
 ['18t', '19t', '13t', '22t', '14t'],
 ['27t', '20t', '81t', '21t', '24t'],
 ['25t', '26t', '79t', '84t', '85t'],
 ['99t', '100t', '46t', '47t', '72t'],
 ['82t', '83t', '88t', '90t', '91t'],
 ['101t', '102t', '28t', '29t', '30t'],
 ['31t', '74t', '32t', '33t', '34t'],
 ['60t', '71t', '77t', '87t', '100t'],
 ['35t', '36t', '37t', '38t', '39t'],
 ['40t', '41t', '57t', '73t', '58t'],
 ['67t', '75t', '68t', '69t', '70t'],
 ['76t', '86t', '92t', '94t', '95t'],
 ['96t', '97t', '98t', '42t', '43t'],
 ['44t', '62t', '63t', '78t', '80t'],
 ['89t', '93t']]

In [14]:
records = []
for group in station_groups:
    # Construct the URL for this group of stations
    url = URL.format(station_ids=','.join(group), measurements=','.join(measurements), sdate=sdate, edate=edate)

    # Fetch data from the API
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())

    # Process the data and add it to the records
    for s in data['stations']:
        for r in s['data']:
            r['DATETIMEDATA'] = records.append(dict(stationID=s['stationID'], **r))

In [15]:
df_Pm10 = pd.DataFrame(records)
df_Pm10

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,02t,2023-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,0.7,49.0,30.0,53.0,754.0,0.0
1,02t,2023-12-01 01:00:00,NaN,NaN,NaN,NaN,NaN,0.8,51.0,29.5,55.0,753.0,0.0
2,02t,2023-12-01 02:00:00,NaN,NaN,NaN,NaN,NaN,0.4,5.0,29.1,57.0,753.0,0.0
3,02t,2023-12-01 03:00:00,NaN,NaN,NaN,NaN,NaN,0.4,30.0,28.5,60.0,753.0,0.0
4,02t,2023-12-01 04:00:00,NaN,NaN,NaN,NaN,NaN,0.7,19.0,28.1,62.0,753.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68986,93t,2024-01-02 20:00:00,19.0,25.0,0.05,1.0,NaN,3.5,63.0,28.9,80.0,754.0,0.0
68987,93t,2024-01-02 21:00:00,26.0,22.0,0.05,1.0,NaN,3.1,51.0,28.4,81.0,755.0,0.0
68988,93t,2024-01-02 22:00:00,22.0,21.0,0.05,1.0,NaN,2.8,43.0,28.1,81.0,755.0,0.0
68989,93t,2024-01-02 23:00:00,20.0,20.0,0.05,1.0,NaN,3.6,37.0,27.9,82.0,755.0,0.0


In [16]:
df_Pm10['DATETIMEDATA'] = df_Pm10['DATETIMEDATA'].str.slice(stop=16)
df_Pm10

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,NaN,0.7,49.0,30.0,53.0,754.0,0.0
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,NaN,0.8,51.0,29.5,55.0,753.0,0.0
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,NaN,0.4,5.0,29.1,57.0,753.0,0.0
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,NaN,0.4,30.0,28.5,60.0,753.0,0.0
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,NaN,0.7,19.0,28.1,62.0,753.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
68986,93t,2024-01-02 20:00,19.0,25.0,0.05,1.0,NaN,3.5,63.0,28.9,80.0,754.0,0.0
68987,93t,2024-01-02 21:00,26.0,22.0,0.05,1.0,NaN,3.1,51.0,28.4,81.0,755.0,0.0
68988,93t,2024-01-02 22:00,22.0,21.0,0.05,1.0,NaN,2.8,43.0,28.1,81.0,755.0,0.0
68989,93t,2024-01-02 23:00,20.0,20.0,0.05,1.0,NaN,3.6,37.0,27.9,82.0,755.0,0.0


In [17]:
df_traffic = pd.read_csv('traffic2023-2024.csv')
df_traffic

,datetime,index
0,2023-01-01 00:00,2.0
1,2023-01-01 00:05,2.6
2,2023-01-01 00:10,1.9
3,2023-01-01 00:15,2.8
4,2023-01-01 00:20,2.5
...,...,...
113219,2024-01-31 23:35,2.1
113220,2024-01-31 23:40,2.3
113221,2024-01-31 23:45,2.7
113222,2024-01-31 23:50,2.6


In [18]:
df_merge = df_Pm10.merge(df_traffic, left_on='DATETIMEDATA', right_on='datetime', how='left')
df_merge

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN,datetime,index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,NaN,0.7,49.0,30.0,53.0,754.0,0.0,2023-12-01 00:00,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,NaN,0.8,51.0,29.5,55.0,753.0,0.0,2023-12-01 01:00,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,NaN,0.4,5.0,29.1,57.0,753.0,0.0,2023-12-01 02:00,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,NaN,0.4,30.0,28.5,60.0,753.0,0.0,2023-12-01 03:00,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,NaN,0.7,19.0,28.1,62.0,753.0,0.0,2023-12-01 04:00,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68986,93t,2024-01-02 20:00,19.0,25.0,0.05,1.0,NaN,3.5,63.0,28.9,80.0,754.0,0.0,2024-01-02 20:00,2.9
68987,93t,2024-01-02 21:00,26.0,22.0,0.05,1.0,NaN,3.1,51.0,28.4,81.0,755.0,0.0,2024-01-02 21:00,3.3
68988,93t,2024-01-02 22:00,22.0,21.0,0.05,1.0,NaN,2.8,43.0,28.1,81.0,755.0,0.0,2024-01-02 22:00,2.5
68989,93t,2024-01-02 23:00,20.0,20.0,0.05,1.0,NaN,3.6,37.0,27.9,82.0,755.0,0.0,2024-01-02 23:00,1.7


In [19]:
df_merge.head(5)

,stationID,DATETIMEDATA,PM10,O3,CO,NO2,SO2,WS,WD,TEMP,RH,BP,RAIN,datetime,index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,NaN,0.7,49.0,30.0,53.0,754.0,0.0,2023-12-01 00:00,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,NaN,0.8,51.0,29.5,55.0,753.0,0.0,2023-12-01 01:00,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,NaN,0.4,5.0,29.1,57.0,753.0,0.0,2023-12-01 02:00,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,NaN,0.4,30.0,28.5,60.0,753.0,0.0,2023-12-01 03:00,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,NaN,0.7,19.0,28.1,62.0,753.0,0.0,2023-12-01 04:00,1.8


In [20]:
df_merge = df_merge.drop(columns=['datetime'])
df_merge.columns = ['stationID', 'datetime', 'pm10', 'o3', 'co', 'no2', 'so2', 'ws', 'wd', 'temp', 'rh', 'bp', 'rain', 'Traffic congestion index']
df_merge

,stationID,datetime,pm10,o3,co,no2,so2,ws,wd,temp,rh,bp,rain,Traffic congestion index
0,02t,2023-12-01 00:00,NaN,NaN,NaN,NaN,NaN,0.7,49.0,30.0,53.0,754.0,0.0,2.4
1,02t,2023-12-01 01:00,NaN,NaN,NaN,NaN,NaN,0.8,51.0,29.5,55.0,753.0,0.0,2.1
2,02t,2023-12-01 02:00,NaN,NaN,NaN,NaN,NaN,0.4,5.0,29.1,57.0,753.0,0.0,2.5
3,02t,2023-12-01 03:00,NaN,NaN,NaN,NaN,NaN,0.4,30.0,28.5,60.0,753.0,0.0,2.3
4,02t,2023-12-01 04:00,NaN,NaN,NaN,NaN,NaN,0.7,19.0,28.1,62.0,753.0,0.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68986,93t,2024-01-02 20:00,19.0,25.0,0.05,1.0,NaN,3.5,63.0,28.9,80.0,754.0,0.0,2.9
68987,93t,2024-01-02 21:00,26.0,22.0,0.05,1.0,NaN,3.1,51.0,28.4,81.0,755.0,0.0,3.3
68988,93t,2024-01-02 22:00,22.0,21.0,0.05,1.0,NaN,2.8,43.0,28.1,81.0,755.0,0.0,2.5
68989,93t,2024-01-02 23:00,20.0,20.0,0.05,1.0,NaN,3.6,37.0,27.9,82.0,755.0,0.0,1.7


In [21]:
df_merge.to_csv('Merged_PM10_data.csv', index=False)